In [4]:
import os
import multiprocessing
import pandas as pd
import numpy as np

from gensim.corpora import WikiCorpus
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence


L = 10

path = "./ccf_data/"
save_path = path + "w2v/"

# 是否存在文件夹
if not os.path.exists(save_path):
    print(save_path)
    os.makedirs(save_path)

# 训练集和测试集
train1 = pd.read_csv(path + "train_all.csv")
train = pd.read_csv(path + "train_2.csv")
test = pd.read_csv(path + "test_2.csv")
print("初赛训练集的形状:{}".format(train1.shape))
print("复赛训练集的形状:{}".format(train.shape))
print("复赛测试集的形状:{}".format(test.shape))

# 将数据集全连接,进行抽样打乱数据集
data = pd.concat([train, test, train1]).reset_index(drop=True).sample(frac=1, random_state=2018).fillna(0)

F:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4,5,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
F:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
F:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


初赛训练集的形状:(743990, 27)
复赛训练集的形状:(374655, 27)
复赛测试集的形状:(160566, 26)


In [5]:
data.head()

,service_type,is_mix_service,online_time,1_total_fee,2_total_fee,3_total_fee,4_total_fee,month_traffic,many_over_bill,contract_type,contract_time,is_promise_low_consume,net_service,pay_times,pay_num,last_month_traffic,local_trafffic_month,local_caller_time,service1_caller_time,service2_caller_time,gender,age,complaint_level,former_complaint_num,former_complaint_fee,current_service,user_id
931737,4,0,180,218.90,229.7,194.95,246.40,824.758124,0,9,12,0,4,1,0.05,1024.000000,553.618685,119.950000,371.083333,433.350000,1,46,0,0,0.0,99999828.0,T6FWM1nqIvt32Voa
1078266,4,0,46,297.91,380.99,359.72,731.15,0.000000,0,1,24,0,4,2,0.01,4096.000000,0.000000,0.000000,0.200000,0.000000,1,0,0,0,0.0,99999825.0,6aFNuTOiWo50Pq8G
141937,1,0,8,53.20,48.2,61.2,60.70,0.000000,1,0,0,0,4,1,50.00,0.000000,3598.514142,0.666667,0.666667,0.000000,2,31,0,0,0.0,90063345.0,gT2J7GQqz3vMOCxS
497963,4,0,34,113.40,96,117.3,174.00,1858.365283,0,3,12,0,4,1,10.00,76.544828,0.000000,0.000000,0.000000,240.483333,1,25,2,1,0.0,0.0,sub5rhlaFHZdKGek
240204,4,0,68,90.00,90,92.6,90.00,4940.241300,0,3,12,1,4,4,200.00,1024.000000,3984.603175,16.950000,0.000000,256.883333,2,28,0,0,0.0,89950167.0,oxpSarLPQtjVFDfe


In [7]:
data = data.replace('\\N', 999)

In [12]:
sentence = []
for line in list(data[["1_total_fee","2_total_fee","3_total_fee","4_total_fee"]].values):
    sentence.append([str(float(l)) for idx, l in enumerate(line)])
    
print("training...")
# sentence格式[[],[],[],[]]
model = Word2Vec(sentence, size=L, window=2, min_count=1, workers=multiprocessing.cpu_count(), iter=10)
print("outputing...")

# 
for fea in ['1_total_fee', '2_total_fee', '3_total_fee', '4_total_fee']:
    values = []
    for line in list(data[fea].values):
        values.append(line)
    values = set(values)
    print(len(values))
    w2v = []
    for i in values:
        a = [i]
        a.extend(model[str(float(i))])
        w2v.append(a)
    out_df = pd.DataFrame(w2v)
    
    name = [fea]
    for i in range(L):
        name.append(name[0] + "W" + str(i))
    out_df.columns = name
    print(out_df.shape)
    out_df.to_csv(save_path + "/" + fea + ".csv", index=False)

training...
outputing...
39983


F:\anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


(39983, 11)
59801
(59801, 11)
55067
(55067, 11)
43735
(43735, 11)
